In [3]:
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [4]:
train = pd.read_csv('data/train-parking.csv', low_memory=False)
test = pd.read_csv('data/test-no-label-parking.csv', low_memory=False)

In [5]:
train.shape

(1100, 8)

In [6]:
train[:2]

,Street,From,To,Date,Time,Real.Spots,Street.Length,any_spot
0,Mission Street,25th Street,26th Street,1/7/2014,16:19,4,179.13297,1
1,Polk Street,Ellis Street,Olive Street,1/18/2014,20:42,0,52.74021,0


In [7]:
test.shape

(726, 6)

In [8]:
test[:2]

,Street,From,To,Date,Time,Street.Length
0,Stockton Street,Stockton Tunnel,Stockton Street,3/28/2014,16:34,35.786472
1,Van Ness Avenue,Hayes Street,Van Ness Avenue,3/28/2014,21:34,63.787968


## EDA based on training data. ##

In [9]:
# drop Real.Spots for now, since it does not exist in test data set.
train = train.drop('Real.Spots', axis=1)

In [10]:
print(train.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 7 columns):
Street           1100 non-null object
From             1100 non-null object
To               1100 non-null object
Date             1100 non-null object
Time             1100 non-null object
Street.Length    1100 non-null float64
any_spot         1100 non-null int64
dtypes: float64(1), int64(1), object(5)
memory usage: 381.5 KB
None


In [11]:
# number of unique street names from Street, From and To these 3 columns.
train_list = list(set().union(train["Street"], train["From"], train["To"])); len(train_list)

87

In [12]:
test_list = list(set().union(test["Street"], test["From"], test["To"])); len(test_list)

87

In [13]:
set(train_list) == set(test_list)

True

Assign each street with an unique ID (index within train_list), and update columns Street, From and to properly.

In [14]:
street_map = {}
for i in range(0, len(train_list)):
    street_map[train_list[i]] = i + 1

In [15]:
# Update train
train = train.replace({"Street": street_map})
train = train.replace({"From": street_map})
train = train.replace({"To": street_map})

In [16]:
# Update test
test = test.replace({"Street": street_map})
test = test.replace({"From": street_map})
test = test.replace({"To": street_map})

In [17]:
# Keep hour part of time
train["Time"] = train["Time"].apply(lambda x : int(x.split(':')[0]))
test["Time"] = test["Time"].apply(lambda x : int(x.split(':')[0]))

In [18]:
# convert Street.Length to integer, I don't think we need float precision for this variable.
# ?? Not sure if this step is neccessary
train["Street.Length"] = train["Street.Length"].apply(lambda x : int(x))
test["Street.Length"] = test["Street.Length"].apply(lambda x : int(x))

In [19]:
# convert Date to datetime 
train["Date"] = pd.to_datetime(train.Date, format = '%m/%d/%Y')
test["Date"] = pd.to_datetime(test.Date, format = '%m/%d/%Y')

In [20]:
# create month column
train["month"] = train.Date.dt.month
test["month"] = test.Date.dt.month

In [21]:
# create dow column
train["dow"] = train.Date.dt.dayofweek
test["dow"] = test.Date.dt.dayofweek

In [22]:
# remove Date column
train = train.drop('Date', axis=1)
test = test.drop('Date', axis=1)

In [23]:
train.head()

,Street,From,To,Time,Street.Length,any_spot,month,dow
0,48,85,11,16,179,1,1,1
1,61,71,34,20,52,0,1,5
2,51,38,47,20,52,0,1,5
3,51,70,22,20,52,0,1,5
4,51,84,66,20,52,0,1,5


In [24]:
# train val split based on time.
X = train.drop("any_spot", axis=1)
y = train["any_spot"].astype('category')
# how to do train & val split ??
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 99)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((880, 7), (220, 7), (880,), (220,))

In [31]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
def f_score(m, X, y):
    preds = m.predict(X)
    p = precision_score(y, preds)
    r = recall_score(y, preds)
    return (1.25*p*r/(0.25*p+r))

## Parameter Tuning ##

In [29]:
# Tune three parameters: n_estimators, min_samepls_leaf, max_features and max_depth
# ?? Will this kind of tuning overfit our validation dataset
numOfestimators = np.arange(25) + 1
numOfleafs = np.arange(15) + 1
numOffeatures = np.arange(0.1, 1.1, 0.1)
depths = np.arange(0, 7) + 1
best_result = []
for numOfestimator in numOfestimators:
    for numOfleaf in numOfleafs:
        for numOffeature in numOffeatures:  
            for depth in depths:
                result = [numOfestimator, numOfleaf, numOffeature, depth]
                m = RandomForestClassifier(n_jobs=-1, n_estimators=numOfestimator,\
                                    min_samples_leaf=numOfleaf,\
                                    max_features=numOffeature, max_depth = depth)
                m.fit(X_train, y_train)
                result.append(f_score(m, X_train, y_train))
                result.append(f_score(m, X_val, y_val))
                if len(best_result) == 0: best_result = result
                elif best_result[5] < result[5]: 
                    print(result)
                    best_result = result
# print(best_result)

[1, 1, 0.10000000000000001, 3, 0.56137012369172212, 0.50314465408805031]


/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


[1, 1, 0.30000000000000004, 5, 0.55823569951757412, 0.52631578947368429]
[1, 1, 0.40000000000000002, 6, 0.61677061677061673, 0.57788944723618085]


/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 d

/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 d

...

/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 d

/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 d

/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/an

/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 d

/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/Users/chengcheng/anaconda/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 d

In [49]:
best_result

[6, 1, 1.0, 0.94658553076402963, 0.64720812182741116]

In [30]:
best_result

[11, 6, 0.90000000000000002, 7, 0.79136690647482011, 0.60734463276836159]

In [29]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=11,\
                            min_samples_leaf=6,\
                            max_features=0.9, max_depth = 7)

In [35]:
m.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features=0.9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [31]:
f_score(m, X_val, y_val)

0.52356020942408377

In [39]:
preds = m.predict(test)

In [40]:
len(preds)

726

In [59]:
def writeToFile(preds, filename):
    result = "id,any_spot"
    for i in range(0, len(preds)):
        result = result + "\n" + str(i + 1) + "," + str(preds[i]) 
    # return result
    file = open(filename,"w")  
    file.write(result)
    file.close()

In [42]:
writeToFile(preds)

## XGboost

In [32]:
import xgboost as xgb

In [37]:
m = xgb.XGBClassifier (max_depth=7, learning_rate=0.1, n_estimators=2, n_jobs=-1)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=1,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [38]:
preds = model.predict(X_val)

In [52]:
depths = np.arange(0, 7) + 1
rates = np.arange(0.1, 1.1, 0.1)
best_f_score = 0
for i in range(1, 300):
    print("i: " + str(i))
    for depth in depths:
        for rate in rates:
            model = xgb.XGBClassifier (max_depth=depth, n_estimators=i, n_jobs=-1,\
                                        learning_rate=rate)
            model.fit(X_train, y_train)
            score = f_score(model, X_val, y_val)
            if score > best_f_score:
                print(score, [depth, i, rate])
                best_f_score = score

i: 1
0.378151260504 [2, 1, 0.10000000000000001]
0.482233502538 [4, 1, 0.10000000000000001]
0.483870967742 [5, 1, 0.10000000000000001]


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_

0.502958579882 [7, 1, 0.10000000000000001]
i: 2
0.503144654088 [3, 2, 0.70000000000000007]
0.530726256983 [4, 2, 0.30000000000000004]
0.534682080925 [4, 2, 0.5]


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_

0.545023696682 [5, 2, 0.80000000000000004]
0.548128342246 [7, 2, 0.90000000000000002]
i: 3


...

0.641025641026 [5, 7, 1.0]
i: 8


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


i: 9


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28
i: 29
i: 30
i: 31
i: 32
i: 33
i: 34
i: 35
i: 36
i: 37
i: 38
i: 39
i: 40
i: 41
i: 42
i: 43
i: 44
i: 45
i: 46
i: 47
i: 48
i: 49
i: 50
i: 51
i: 52
i: 53
i: 54
i: 55
i: 56
i: 57
i: 58
i: 59
i: 60
i: 61
i: 62
i: 63
i: 64
i: 65
i: 66
i: 67
i: 68
i: 69
i: 70
i: 71
i: 72
i: 73
i: 74
i: 75
i: 76
i: 77
i: 78
i: 79
i: 80
i: 81
i: 82
i: 83
i: 84
i: 85
i: 86
i: 87
i: 88
i: 89
i: 90
i: 91
i: 92
i: 93
i: 94
i: 95
i: 96
i: 97
i: 98
i: 99
i: 100
i: 101
i: 102
i: 103
i: 104
i: 105
i: 106
i: 107
i: 108
i: 109
i: 110
i: 111
i: 112
i: 113
i: 114
i: 115
i: 116
i: 117
i: 118
i: 119
i: 120
i: 121
i: 122
i: 123
i: 124
i: 125
i: 126
i: 127
i: 128
i: 129
i: 130
i: 131
i: 132
i: 133
i: 134
i: 135
i: 136
i: 137
i: 138
i: 139
i: 140
i: 141
i: 142
i: 143
i: 144
i: 145
i: 146
i: 147
i: 148
i: 149
i: 150
i: 151
i: 152
i: 153
i: 154
i: 155
i: 156
i: 157
i: 158
i: 159
i: 160
i: 161
i: 162
i: 163
i: 164
i: 16

In [49]:
model = xgb.XGBClassifier (max_depth=3, learning_rate=0.1, n_estimators=262, n_jobs=-1)
model.fit(X_train, y_train)
f_score(model, X_val, y_val)

0.59241706161137442

In [44]:
preds = model.predict(test)

In [56]:
model = xgb.XGBClassifier (max_depth=5, learning_rate=1, n_estimators=7, n_jobs=-1)
model.fit(X_train, y_train)
f_score(model, X_val, y_val)

0.64102564102564108

In [57]:
preds = model.predict(test);preds

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0,

In [58]:
len(preds)

726

In [60]:
writeToFile(preds, "submission1.csv")

In [61]:
!ls

archive  parkSF.ipynb	     README.md	report_notes.md  submission1.csv
data	 parse_sensor.ipynb  ref_data	scraping_nb	 tims_eda_nb.ipynb


In [62]:
!pwd

/home/ec2-user/final-project-chengcheng-tim
